In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('DeadFuelLag').getOrCreate()

In [15]:
# import pandas as pd
# df = pd.read_parquet('gridMet.parquet.gz')
# #df["surface_downwelling_shortwave_flux_in_air_W_m-2"] = df["surface_downwelling_shortwave_flux_in_air_W m-2"]
# #del df["surface_downwelling_shortwave_flux_in_air_W m-2"]
# df["wind_from_direction_Degrees_Clockwise_from_north"] = df["wind_from_direction_Degrees Clockwise_from_north"]
# del df["wind_from_direction_Degrees Clockwise_from_north"]
# df.to_parquet('gridMet.parquet.gz')

In [14]:
df = spark.read.parquet("gridMet.parquet.gz").where(F.col("precipitation_amount_mm").isNotNull())

In [15]:
df.printSchema()

root
 |-- precipitation_amount_mm: double (nullable = true)
 |-- relative_humidity_%: double (nullable = true)
 |-- specific_humidity_kg/kg: double (nullable = true)
 |-- wind_speed_m/s: double (nullable = true)
 |-- max_air_temperature_K: double (nullable = true)
 |-- min_air_temperature_K: double (nullable = true)
 |-- burning_index_g_Unitless: double (nullable = true)
 |-- dead_fuel_moisture_100hr_Percent: double (nullable = true)
 |-- dead_fuel_moisture_1000hr_Percent: double (nullable = true)
 |-- energy_release_component-g_Unitless: double (nullable = true)
 |-- potential_evapotranspiration_mm: double (nullable = true)
 |-- mean_vapor_pressure_deficit_kPa: double (nullable = true)
 |-- surface_downwelling_shortwave_flux_in_air_W_m-2: double (nullable = true)
 |-- wind_from_direction_Degrees_Clockwise_from_north: double (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [20]:
windowSpec = Window.partitionBy(F.col("latitude"), F.col("longitude")).orderBy(F.col("date").desc())
df = df.withColumnRenamed("surface_downwelling_shortwave_flux_in_air_W_m-2","surface_downwelling_shortwave_flux_in_air_W m-2")
df = df.withColumn("cumLag", F.lit(0))
for i in range(1,8):
    df = df.withColumn("lag-"+str(i),F.col("dead_fuel_moisture_1000hr_Percent")- F.lag(F.col("dead_fuel_moisture_1000hr_Percent"), i).over(windowSpec))
    df = df.withColumn("cumLag", F.when((F.col("lag-"+str(i))<= 0) & (F.col("cumLag") == i-1), i ).otherwise(F.col("cumLag")))
df = df.withColumn("lag-Test",F.lag(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec))

In [24]:
df = df.withColumn( "movingAvg",F.avg(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-7,0)))

In [25]:
df.take(20)

[Row(precipitation_amount_mm=0.0, relative_humidity_%=27.200000000000003, specific_humidity_kg/kg=0.005090000000000001, wind_speed_m/s=1.8, max_air_temperature_K=292.6, min_air_temperature_K=274.2, burning_index_g_Unitless=17.0, dead_fuel_moisture_100hr_Percent=19.1, dead_fuel_moisture_1000hr_Percent=22.1, energy_release_component-g_Unitless=19.0, potential_evapotranspiration_mm=1.7000000000000002, mean_vapor_pressure_deficit_kPa=0.67, surface_downwelling_shortwave_flux_in_air_W m-2=121.4, wind_from_direction_Degrees_Clockwise_from_north=92.0, date=datetime.datetime(2019, 12, 30, 16, 0), latitude=33.025000000000006, longitude=-116.97499996666667, cumLag=0, lag-1=None, lag-2=None, lag-3=None, lag-4=None, lag-5=None, lag-6=None, lag-7=None, lag-Test=None, movingAvg=22.1),
 Row(precipitation_amount_mm=0.0, relative_humidity_%=52.900000000000006, specific_humidity_kg/kg=0.005960000000000001, wind_speed_m/s=2.6, max_air_temperature_K=286.8, min_air_temperature_K=279.9, burning_index_g_Unitl